In [15]:
from typing import List

In [145]:
import numpy as np

def ZDT1(x):
    n = len(x)  # number of decision variables
    f1 = x[0]  # objective 1 is just the first variable
    g = 1 + 9 * np.sum(x[1:]) / (n - 1)
    f2 = g * (1 - np.sqrt(f1 / g))  # objective 2 formula
    return f1, f2


In [146]:
import random


class Individual:
    """
    Individual of genetic algorithm
    code - represents one solution
    fitness - quality of the solution
    """
    def __init__(self, num_variables):
        self.code = np.random.uniform(0, 1, num_variables) #TODO change for different objectives
        self.rank = None
        self.crowding_distance = None
        self.calc_fitness()

    def calc_fitness(self):
        f1,f2 = ZDT1(self.code)
        self.fitness = (f1, f2)
        
        
    # TODO check if i need it for selection and elitism
#     def __gt__(self, other):
#         """
#         Defines the greater-than comparison for Individuals based on rank and crowding distance.
#         """
#         if self.rank is None or other.rank is None:
#             raise ValueError("Rank must be assigned before comparison.")
#         if self.crowding_distance is None or other.crowding_distance is None:
#             raise ValueError("Crowding distance must be assigned before comparison.")
        
#         # Compare based on rank and then crowding distance
#         if self.rank < other.rank:
#             return True
#         elif self.rank == other.rank:
#             return self.crowding_distance > other.crowding_distance
#         else:
#             return False

In [147]:
def dominates(first_individual, second_individual):
    is_better_in_all = True
    is_strictly_better = False

    for f1, f2 in zip(first_individual.fitness, second_individual.fitness):
        if f1 > f2:
            is_better_in_all = False
        if f1 < f2:
            is_strictly_better = True

    return is_better_in_all and is_strictly_better
    

In [148]:
def non_dominated_sorting(population: List[Individual]):
    pareto_fronts = []
    domination_count = {}  # Stores how many individuals dominate each individual
    dominated_individuals = {}  # Stores the individuals dominated by each individual
    n = len(population)
    
    for individual in population:
        domination_count[individual] = 0
        dominated_individuals[individual] = []
    
    for i in range(n):
        for j in range(n):
            if i!=j:
                if dominates(population[i],population[j]):
                    dominated_individuals[population[i]].append(population[j])
                elif dominates(population[j],population[i]):
                    domination_count[population[i]] += 1
     
    current_front = [ind for ind in population if domination_count[ind] == 0]
    current_front_index = 1
    while current_front:
        pareto_fronts.append(current_front)
        next_front = []
        for individual in current_front:
            individual.rank = current_front_index
            for dominated in dominated_individuals[individual]:
                domination_count[dominated] -= 1
                if domination_count[dominated] == 0:
                    next_front.append(dominated)
        current_front = next_front
        current_front_index += 1

    return pareto_fronts

In [149]:
from copy import deepcopy

def calculate_crowding_distance(pareto_front: List[Individual], num_objectives: int):
    n = len(pareto_front)
    # Firstly initialize every individual from front to have cd = 0
    for ind in pareto_front:
        ind.crowding_distance = 0
    for i in range(num_objectives):
        # Sort individuals based on objective
        pareto_front.sort(key = lambda x : x.fitness[i])
        pareto_front[0].crowding_distance = float('inf')
        pareto_front[-1].crowding_distance = float('inf')
        f_min = pareto_front[0].fitness[i]
        f_max = pareto_front[-1].fitness[i]
        for k in range(1,n-1):
            assert f_max != f_min
            distance = (pareto_front[k+1].fitness[i] - pareto_front[k-1].fitness[i])/(f_max - f_min)
            pareto_front[k].crowding_distance += distance
        
    

In [150]:
def selection(population: List[Individual], k: int):
    """
    Tournament selection
    1. Choose random k individuals from the population
    2. The best of those k wins the tournament and is selected for crossover

    Primarni kriterijum za selekciju je rang. 
    Ako dve jedinke imaju isti rang, 
    preferira se ona sa većom distancom gužve.
    """
    k = min(len(population), k)
    participants = random.sample(population, k)
    
    return max(participants, key=lambda x: (-x.rank, x.crowding_distance))


In [151]:
def crossover(parent1: Individual, parent2: Individual, child1: Individual, child2: Individual):
    """
    Single point crossover - changes children in place
    TODO SBX
    """
    breakpoint = random.randrange(1, len(parent1.code))
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child1.code[breakpoint:] = parent2.code[breakpoint:]

    child2.code[:breakpoint] = parent2.code[:breakpoint]
    child2.code[breakpoint:] = parent1.code[breakpoint:]
    

In [152]:
def mutation(child: Individual, p: float):
    """
    TODO polynomial mutation
    """
    for i in range(len(child.code)):
        if random.random() < p:
            child.code[i] = 1 - child.code[i]

testing sorting:

In [153]:
population = [Individual(num_variables=30) for _ in range(10)]
for ind in population:
    ind.fitness = ZDT1(ind.code)  # Calculate fitness for ZDT1
# Perform non-dominated sorting
pareto_fronts = non_dominated_sorting(population)
# Print results
for i, front in enumerate(pareto_fronts):
    print(f"front {i + 1}:")
    calculate_crowding_distance(front,2)
    for ind in front:
        print(f" fitness:{ind.fitness}")
        
population.sort(key = lambda x : x.rank)
for ind in population:
    print(ind.rank,'--',ind.fitness,'--',ind.crowding_distance)

front 1:
 fitness:(0.6660042077659039, 3.583990291519061)
 fitness:(0.2894854102597064, 3.7598464413407244)
 fitness:(0.0100015457342848, 4.643712057515177)
front 2:
 fitness:(0.3043042398597151, 3.804054380900897)
 fitness:(0.020614451749967455, 5.099515173104652)
front 3:
 fitness:(0.5651134148981395, 3.8216085669031425)
 fitness:(0.44441838336501993, 3.85718545142365)
 fitness:(0.3347898894967698, 4.233190345208425)
front 4:
 fitness:(0.825469493748813, 3.9859670288053395)
 fitness:(0.4461176468103628, 4.532520725686009)
1 -- (0.0100015457342848, 4.643712057515177) -- inf
1 -- (0.2894854102597064, 3.7598464413407244) -- 2.0
1 -- (0.6660042077659039, 3.583990291519061) -- inf
2 -- (0.3043042398597151, 3.804054380900897) -- inf
2 -- (0.020614451749967455, 5.099515173104652) -- inf
3 -- (0.44441838336501993, 3.85718545142365) -- 2.0
3 -- (0.5651134148981395, 3.8216085669031425) -- inf
3 -- (0.3347898894967698, 4.233190345208425) -- inf
4 -- (0.825469493748813, 3.9859670288053395) -- in